# Import libraries

In [1]:
import pandas as pd # data preprocessing
import itertools # confusion matrix
import string
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import hamming_loss, accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline
# To show all the rows of pandas dataframe
pd.set_option('display.max_rows', None)
import spacy

In [2]:
df = pd.read_csv(r"D:\DATA SCEINCE\1. GL\9. FINAL SORTED PROJECT\Final_NLP_projects\Drugs.csv")

In [4]:
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,"November 27, 2016",37


In [97]:
df.condition.value_counts()[:10]

Birth Control      28788
Depression          9069
Pain                6145
Anxiety             5904
Acne                5588
Bipolar Disorde     4224
Insomnia            3673
Weight Loss         3609
Obesity             3568
ADHD                3383
Name: condition, dtype: int64

In [84]:
df.drugName.value_counts()[:10]

Levonorgestrel                        3631
Etonogestrel                          3321
Ethinyl estradiol / norethindrone     2750
Nexplanon                             2156
Ethinyl estradiol / norgestimate      2033
Ethinyl estradiol / levonorgestrel    1809
Phentermine                           1538
Sertraline                            1353
Escitalopram                          1287
Mirena                                1242
Name: drugName, dtype: int64

In [100]:
df.usefulCount.sort_values().head(10)

58957     0
127103    0
65615     0
127098    0
144989    0
65660     0
65664     0
65670     0
65684     0
127054    0
Name: usefulCount, dtype: int64

In [16]:
print('Number of drugs found are:' ,len(df.drugName.value_counts()))
print('Number of conditions found to be:',len(df.condition.value_counts()))

Number of drugs found are: 3436
Number of conditions found to be: 884


In [17]:
#multiple drugs are used for same condition 

In [98]:
df[df['drugName']=='Clonazepam'].head(10)

,drugName,condition,review,rating,usefulCount,cleaned_review
42,Clonazepam,Panic Disorde,"""This medication changed my life. My panic at...",9,30,This medication changed my life My panic atta...
448,Clonazepam,Anxiety,"""Many people are unreasonably scared of Klonop...",10,261,Many people are unreasonably scared of Klonopi...
523,Clonazepam,Panic Disorde,"""I have PTSD, OCD, severe bipolar and severe b...",7,24,I have PTSD OCD severe bipolar and severe bord...
558,Clonazepam,Night Terrors,"""I had been taking anti-anxiety medication for...",5,9,I had been taking antianxiety medication for a...
771,Clonazepam,Anxiety,"""I have been on this anxiolytic longer than an...",9,47,I have been on this anxiolytic longer than any...
841,Clonazepam,Panic Disorde,"""Clonazepam has drastically improved my life. ...",10,8,Clonazepam has drastically improved my life I ...
877,Clonazepam,Insomnia,"""I have to weigh in about this med. When you ...",1,49,I have to weigh in about this med When you fi...
1118,Clonazepam,Panic Disorde,"""Helped me more than other benzodiapines.""",9,13,Helped me more than other benzodiapines
1171,Clonazepam,Anxiety,"""I have been taking clonazepam .5 MG&#039;s on...",10,20,I have been taking clonazepam MGs once daily ...
1342,Clonazepam,Insomnia,"""Don&#039;t start this drug for sleep or anxie...",1,33,Dont start this drug for sleep or anxiety Its ...


In [9]:
df.isnull().sum()

Unnamed: 0       0
drugName         0
condition      899
review           0
rating           0
date             0
usefulCount      0
dtype: int64

In [39]:
df.drop(['Unnamed: 0','date'],axis=1,inplace=True)

In [19]:
df.reset_index(inplace=True)

In [13]:
df.dropna(inplace=True)

In [21]:
df.drop('index',inplace=True,axis=1)

In [15]:
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
import unicodedata

In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160398 entries, 0 to 161296
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   drugName        160398 non-null  object
 1   condition       160398 non-null  object
 2   review          160398 non-null  object
 3   rating          160398 non-null  int64 
 4   usefulCount     160398 non-null  int64 
 5   cleaned_review  160398 non-null  object
dtypes: int64(2), object(4)
memory usage: 12.6+ MB


In [14]:
# clean the data
def remove_special_characters(text):
    pattern = r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [7]:
df['cleaned_review']=df['review'].apply(remove_special_characters)

In [110]:
df.head()

,drugName,condition,review,rating,usefulCount,cleaned_review
0,Valsartan,Left Ventricular Dysfunction,"It has no side effect, I take it in combinatio...",9,27,It has no side effect I take it in combination...
1,Guanfacine,ADHD,My son is halfway through his fourth week of I...,8,192,My son is halfway through his fourth week of I...
2,Lybrel,Birth Control,"I used to take another oral contraceptive, whi...",5,17,I used to take another oral contraceptive whic...
3,Ortho Evra,Birth Control,This is my first time using any form of birth ...,8,10,This is my first time using any form of birth ...
4,Buprenorphine / naloxone,Opiate Dependence,Suboxone has completely turned my life around....,9,37,Suboxone has completely turned my life around ...


# Sentiment analysis using Afinn

In [16]:
from afinn import Afinn

In [17]:
af = Afinn()

In [14]:
sentiment_scores=[af.score(article) for article in df.cleaned_review]

NameError: name 'af' is not defined

In [142]:
sentiment_category = ['positive' if score > 0 
                          else 'negative' if score < 0 
                              else 'neutral' 
                                  for score in sentiment_scores]

In [206]:
af = pd.DataFrame([list(df['cleaned_review']), sentiment_scores, sentiment_category]).T
af.columns = ['review', 'sentiment_score', 'sentiment_category']
#sf['sentiment_score'] = df.sentiment_score.astype('float')


In [150]:
af.head()

,review,sentiment_score,sentiment_category
0,It has no side effect I take it in combination...,-1.0,negative
1,My son is halfway through his fourth week of I...,6.0,positive
2,I used to take another oral contraceptive whic...,1.0,positive
3,This is my first time using any form of birth ...,5.0,positive
4,Suboxone has completely turned my life around ...,7.0,positive


In [157]:
af[af['sentiment_category']=='negative'].head()

,review,sentiment_score,sentiment_category
0,It has no side effect I take it in combination...,-1.0,negative
5,nd day on mg started to work with rock hard er...,-1.0,negative
7,Abilify changed my life There is hope I was on...,-7.0,negative
8,I Ve had nothing but problems with the Keppe...,-4.0,negative
12,I had Crohns with a resection years ago and h...,-7.0,negative


In [158]:
af['review'][7]

'Abilify changed my life There is hope I was on Zoloft and Clonidine when I first started Abilify at the age of  Zoloft for depression and Clondine to manage my complete rage My moods were out of control I was depressed and hopeless one second and then mean irrational and full of rage the next My Dr prescribed me mg of Abilify and from that point on I feel like I have been cured though I know Im not Bipolar disorder is a constant battle I know Abilify works for me because I have tried to get off it and lost complete control over my emotions Went back on it and I was golden again  I am on mg x daily I am now  and better than I have ever been in the past Only side effect is I like to eat a lot'

# Sentiment analysis using TextBlob

In [8]:
from textblob import TextBlob

In [22]:
sentiment_scores_tb = [round(TextBlob(article).sentiment.polarity, 3) for article in df.cleaned_review]

In [23]:
sentiment_category_tb = ['positive' if score > 0 
                          else 'negative' if score < 0 
                              else 'neutral' 
                                  for score in sentiment_scores_tb]

In [ ]:
tf = pd.DataFrame([list(df['cleaned_review']), sentiment_scores_tb, sentiment_category_tb]).T
tf.columns = ['review', 'sentiment_score_tb', 'sentiment_category']
#sf['sentiment_score'] = df.sentiment_score.astype('float')

In [215]:
tf.head()

,review,sentiment_score_tb,sentiment_category
0,It has no side effect I take it in combination...,0.0,neutral
1,My son is halfway through his fourth week of I...,0.168,positive
2,I used to take another oral contraceptive whic...,0.067,positive
3,This is my first time using any form of birth ...,0.18,positive
4,Suboxone has completely turned my life around ...,0.194,positive


# Sentiment analysis using Vader

In [118]:
pip install vaderSentiment


     -------------------------------------- 126.0/126.0 kB 1.2 MB/s eta 0:00:00


In [119]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [120]:
analyzer = SentimentIntensityAnalyzer()

In [122]:
df['rating'] = df['cleaned_review'].apply(analyser.polarity_scores)

In [124]:
vf=df[['review','rating']]

In [126]:
vf.tail()

,review,rating
160393,"""I wrote my first report in Mid-October of 201...","{'neg': 0.038, 'neu': 0.815, 'pos': 0.147, 'co..."
160394,"""I was given this in IV before surgey. I immed...","{'neg': 0.144, 'neu': 0.766, 'pos': 0.089, 'co..."
160395,"""Limited improvement after 4 months, developed...","{'neg': 0.484, 'neu': 0.376, 'pos': 0.141, 'co..."
160396,"""I&#039;ve been on thyroid medication 49 years...","{'neg': 0.052, 'neu': 0.821, 'pos': 0.128, 'co..."
160397,"""I&#039;ve had chronic constipation all my adu...","{'neg': 0.027, 'neu': 0.786, 'pos': 0.187, 'co..."


# Comparing the both 

In [216]:
tf[tf.sentiment_category=='neutral'].head()

,review,sentiment_score_tb,sentiment_category
0,It has no side effect I take it in combination...,0.0,neutral
24,Do not use the cream that comes with this It t...,0.0,neutral
44,At initial testing my VL was over million I r...,0.0,neutral
53,My mother died from lung cancer Her last hope ...,0.0,neutral
67,it caused me to gain pounds,0.0,neutral


In [221]:
tf[tf.sentiment_category=='negative'].head()

,review,sentiment_score_tb,sentiment_category
12,I had Crohns with a resection years ago and h...,-0.107,negative
13,Have a little bit of a lingering cough from a ...,-0.037,negative
19,I have been on Tasigna for just over years no...,-0.111,negative
31,I am years old I had a multiple composite spi...,-0.228,negative
32,The burning is out of control about minutes a...,-1.0,negative


In [228]:
af[af.sentiment_category=='negative'].head()

,review,sentiment_score,sentiment_category,text_lemmatized
0,It has no side effect I take it in combination...,-1.0,negative,"[It, ha, no, side, effect, I, take, it, in, co..."
5,nd day on mg started to work with rock hard er...,-1.0,negative,"[nd, day, on, mg, started, to, work, with, roc..."
7,Abilify changed my life There is hope I was on...,-7.0,negative,"[Abilify, changed, my, life, There, is, hope, ..."
8,I Ve had nothing but problems with the Keppe...,-4.0,negative,"[I, Ve, had, nothing, but, problem, with, the,..."
12,I had Crohns with a resection years ago and h...,-7.0,negative,"[I, had, Crohns, with, a, resection, year, ago..."


# create a new dataframe for classification of condition

In [51]:
df1=pd.concat((df[['condition','drugName','rating','usefulCount']],tf), axis=1)

In [52]:
df1.head()

,condition,drugName,rating,usefulCount,review,sentiment_score_tb,sentiment_category
0,Left Ventricular Dysfunction,Valsartan,9,27,It has no side effect I take it in combination...,0.0,neutral
1,ADHD,Guanfacine,8,192,My son is halfway through his fourth week of I...,0.168,positive
2,Birth Control,Lybrel,5,17,I used to take another oral contraceptive whic...,0.067,positive
3,Birth Control,Ortho Evra,8,10,This is my first time using any form of birth ...,0.18,positive
4,Opiate Dependence,Buprenorphine / naloxone,9,37,Suboxone has completely turned my life around ...,0.194,positive


In [53]:
clf_df=df1[['condition','review']]

In [54]:
clf_df.isnull().sum()

condition    0
review       0
dtype: int64

In [55]:
clf_df.head()

,condition,review
0,Left Ventricular Dysfunction,It has no side effect I take it in combination...
1,ADHD,My son is halfway through his fourth week of I...
2,Birth Control,I used to take another oral contraceptive whic...
3,Birth Control,This is my first time using any form of birth ...
4,Opiate Dependence,Suboxone has completely turned my life around ...


# lemmatization

In [75]:
nlp = spacy.load('en_core_web_sm')

In [57]:
import spacy
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
import unicodedata

In [76]:
def lemmatize_txt(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [82]:
lemmatize_txt((clf_df.review[2]))

'I use to take another oral contraceptive which have   pill cycle and be very happy very light period max   day no other side effect but it contain hormone gestodene which be not available in US so I switch to Lybrel because the ingredient be similar when my other pill end I start Lybrel immediately on my first day of period as the instruction say and the period last for two week when take the second pack same two week and now with third pack thing get even bad my third period last for two week and now its the end of the third week I still have daily brown discharge \r\n the positive side be that I do not have any other side effect the idea of be period free be so tempting Alas'

In [61]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()


In [78]:
#stemming
#use this
def lemmatize_text(text):
    l_words=[lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    return (' '.join(l_words))

In [80]:
lemmatize_text((clf_df.review[2]))

'I used to take another oral contraceptive which had pill cycle and wa very happy very light period max day no other side effect But it contained hormone gestodene which is not available in US so I switched to Lybrel because the ingredient are similar When my other pill ended I started Lybrel immediately on my first day of period a the instruction said And the period lasted for two week When taking the second pack same two week And now with third pack thing got even worse my third period lasted for two week and now it the end of the third week I still have daily brown discharge The positive side is that I didnt have any other side effect The idea of being period free wa so tempting Alas'

# Stop Words

In [56]:
from nltk.corpus import stopwords

stop = stopwords.words('english')

In [58]:
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

In [68]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [70]:
remove_stopwords((clf_df.review[1]))

'My son halfway fourth week Intuniv We became concerned began last week started taking highest dose For two days could hardly get bed cranky slept nearly hours drive home school vacation unusual I called doctor Monday morning said stick days See school getting morning The last two days problem free He MUCH agreeable ever He less emotional good thing less cranky He remembering things Overall behavior better We tried many different medications far effective'

# apply lemma and stop words

In [83]:
clf_df['l_review']=clf_df.review.apply(lemmatize_text)

C:\Users\user\AppData\Local\Temp\ipykernel_3904\22338487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_df['l_review']=clf_df.review.apply(lemmatize_text)


In [86]:
clf_df['final_review']=clf_df['l_review'].apply(remove_stopwords)

C:\Users\user\AppData\Local\Temp\ipykernel_3904\22088370.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_df['final_review']=clf_df['l_review'].apply(remove_stopwords)


In [87]:
clf_df.head()

,condition,review,l_review,final_review
0,Left Ventricular Dysfunction,It has no side effect I take it in combination...,It ha no side effect I take it in combination ...,It ha no side effect I take combination Bystol...
1,ADHD,My son is halfway through his fourth week of I...,My son is halfway through his fourth week of I...,My son halfway fourth week Intuniv We became c...
2,Birth Control,I used to take another oral contraceptive whic...,I used to take another oral contraceptive whic...,I used take another oral contraceptive pill cy...
3,Birth Control,This is my first time using any form of birth ...,This is my first time using any form of birth ...,This first time using form birth control Im gl...
4,Opiate Dependence,Suboxone has completely turned my life around ...,Suboxone ha completely turned my life around I...,Suboxone ha completely turned life around I fe...


# Prepare for model building

In [90]:
y.value_counts()

Birth Control                                                          28788
Depression                                                              9069
Pain                                                                    6145
Anxiety                                                                 5904
Acne                                                                    5588
Bipolar Disorde                                                         4224
Insomnia                                                                3673
Weight Loss                                                             3609
Obesity                                                                 3568
ADHD                                                                    3383
Diabetes, Type 2                                                        2554
Emergency Contraception                                                 2463
High Blood Pressure                                                     2321

In [105]:
counts=clf_df['condition'].value_counts()

In [107]:
to_save=counts[counts>=2000].index

In [110]:
ml_df=clf_df[clf_df.condition.isin(to_save)]

In [113]:
ml_df.shape,df.shape

((85659, 4), (160398, 8))

In [281]:
ml_df.head()

,condition,review,l_review,final_review
0,ADHD,My son is halfway through his fourth week of I...,My son is halfway through his fourth week of I...,My son halfway fourth week Intuniv We became c...
1,Birth Control,I used to take another oral contraceptive whic...,I used to take another oral contraceptive whic...,I used take another oral contraceptive pill cy...
2,Birth Control,This is my first time using any form of birth ...,This is my first time using any form of birth ...,This first time using form birth control Im gl...
3,Emergency Contraception,He pulled out but he cummed a bit in me I took...,He pulled out but he cummed a bit in me I took...,He pulled cummed bit I took Plan B hour later ...
4,Bipolar Disorde,Abilify changed my life There is hope I was on...,Abilify changed my life There is hope I wa on ...,Abilify changed life There hope I wa Zoloft Cl...


In [128]:
# to predict 15 class of disease

In [280]:
ml_df.drop('index',inplace=True,axis=1)

C:\Users\user\AppData\Local\Temp\ipykernel_3904\3661400728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ml_df.drop('index',inplace=True,axis=1)


In [283]:
X=ml_df['final_review']
y=ml_df['condition']

In [132]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.2, random_state=42)

In [327]:
cv= CountVectorizer(stop_words='english',max_df=0.75,ngram_range=(1, 2),max_features=20000)

cv_train = cv.fit_transform(X_train)

cv_test = cv.transform(X_test)

In [328]:
mnb = MultinomialNB()
mnb.fit(cv_train, y_train)
pred1 = mnb.predict(cv_test)
score = metrics.accuracy_score(y_test, pred1)
print("accuracy:   %0.3f" % score) 

accuracy:   0.838


In [332]:
hamming_loss(y_test, pred1)

0.16221106700910576

In [335]:
text=['i feel very cranky and sleepy all day, i dont feel like waking up, i am angry and lazy']
hub=cv.transform(text) #.reshape(1, -1)

In [336]:
mnb.predict(hub)

array(['Depression'], dtype='<U25')

In [138]:
from sklearn.neighbors import KNeighborsClassifier

In [139]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(cv_train, y_train)
pred = knn.predict(cv_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score) 

C:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


accuracy:   0.670


In [173]:
hamming_loss(y_test, pred)

0.3904389446649545

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [329]:
tidf= TfidfVectorizer(stop_words='english', max_features=15000, max_df=0.8,ngram_range=(1,2))
t_train = tidf.fit_transform(X_train)

t_test = tidf.transform(X_test)

In [330]:
mnbtf = MultinomialNB()
mnbtf.fit(t_train, y_train)
pred2 = mnbtf.predict(t_test)
score = metrics.accuracy_score(y_test, pred2)
print("accuracy:   %0.3f" % score) 

accuracy:   0.813


In [331]:
hamming_loss(y_test, pred2)

0.18748540742470232

In [338]:
text_tif=['i feel very cranky and sleepy all day, i dont feel like waking up, i am angry and lazy']
hub_tif=tidf.transform(text) #.reshape(1, -1)

In [340]:
result_tif=mnbtf.predict(hub_tif)

In [350]:
df[df['condition']==result_tif[0]][['drugName','usefulCount']].sort_values('usefulCount',ascending=False).head(5)

,drugName,usefulCount
6674,Sertraline,1291
33349,Zoloft,1291
145314,Zoloft,949
151979,Celexa,771
2028,Citalopram,771


In [352]:
def most_informative_feature_for_class(vectorizer, classifier, classlabel, n=10):
    labelid = list(classifier.classes_).index(classlabel)
    feature_names = vectorizer.get_feature_names()
    topn = sorted(zip(classifier.coef_[labelid], feature_names))[-n:]

    for coef, feat in topn:
        print (classlabel, feat, coef)

In [354]:
most_informative_feature_for_class(tidf, mnbtf, 'Birth Control')

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Birth Control year -5.642591673248187
Birth Control im -5.596193596173997
Birth Control ive -5.437212918733357
Birth Control control -5.424210406899902
Birth Control birth control -5.418022087595799
Birth Control birth -5.391730981203909
Birth Control pill -5.214398203464046
Birth Control month -5.121786245221603
Birth Control wa -5.019944231472163
Birth Control period -4.927941295142861


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


In [358]:
most_informative_feature_for_class(tidf, mnbtf, 'Insomnia')

Insomnia wa -5.994056776940453
Insomnia wake -5.899149543749816
Insomnia work -5.865863050093098
Insomnia insomnia -5.587355240391938
Insomnia hour -5.553364960248255
Insomnia mg -5.433817837597111
Insomnia asleep -5.398069723985971
Insomnia ambien -5.308346544199882
Insomnia night -5.154506666798633
Insomnia sleep -4.545230283325494


In [359]:
most_informative_feature_for_class(tidf, mnbtf, 'Depression')

Depression taking -5.801802734143334
Depression week -5.794780246855711
Depression year -5.786519287160886
Depression day -5.753760333081609
Depression anxiety -5.682493718436265
Depression effect -5.640592801065457
Depression feel -5.560968186115868
Depression wa -5.295880096112205
Depression mg -5.236418553493874
Depression depression -5.103048735171846


In [360]:
most_informative_feature_for_class(tidf, mnbtf, 'High Blood Pressure')

High Blood Pressure year -6.222472685881216
High Blood Pressure medicine -6.094963520591389
High Blood Pressure wa -6.069934239931852
High Blood Pressure taking -6.0605831460652615
High Blood Pressure mg -5.929254702017925
High Blood Pressure effect -5.784777286909223
High Blood Pressure bp -5.744406779766838
High Blood Pressure blood -5.002000686993628
High Blood Pressure blood pressure -4.879648973323476
High Blood Pressure pressure -4.833680059451859


# trying from cosine similarity method

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
cf=df[['condition','review']]

In [7]:
cf.dropna(inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_5944\2461326213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf.dropna(inplace=True)


In [8]:
cf.isnull().sum()

condition    0
review       0
dtype: int64

In [16]:
cf['cleaned_review']=cf['review'].apply(remove_special_characters)

C:\Users\user\AppData\Local\Temp\ipykernel_5944\946041399.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf['cleaned_review']=cf['review'].apply(remove_special_characters)


In [32]:
counts=cf['condition'].value_counts()

to_save_df=counts[counts>=3382].index

cs_df=cf[cf.condition.isin(to_save_df)]

cs_df.shape,df.shape

((73951, 3), (161297, 7))

In [31]:
cf['condition'].value_counts().head(10)

Birth Control      28788
Depression          9069
Pain                6145
Anxiety             5904
Acne                5588
Bipolar Disorde     4224
Insomnia            3673
Weight Loss         3609
Obesity             3568
ADHD                3383
Name: condition, dtype: int64

In [33]:
cd_df.shape

(85659, 3)

In [35]:
combined_features=cs_df['condition']+' '+cs_df['cleaned_review']

In [36]:
len(combined_features)

73951

In [38]:
tidf= TfidfVectorizer(stop_words='english', max_features=10000, max_df=0.8,ngram_range=(1,2))
tf_vects = tidf.fit_transform(combined_features)


In [41]:
tf_vects.shape

(73951, 10000)

In [42]:
simlilarity=cosine_similarity(tf_vects)

MemoryError: Unable to allocate 32.6 GiB for an array with shape (4376220235,) and data type int64

# using tensor flow

In [177]:
 from tensorflow.keras.layers import Dense,TextVectorization

In [175]:
import tensorflow as tf

In [195]:
vect=TextVectorization(output_sequence_length=1800,max_tokens=20000) #after reaching 1800 it will padd out the data

In [210]:
vect.adapt(X.values)


In [198]:
vect.get_weights()

[array([b'i', b'wa', b'day', ..., b'inguinal', b'ingredinets',
        b'ingredientsquot'], dtype=object)]

In [ ]:
vect.get_vocabulary()

In [211]:
vect_txt=vect(X.values) #tokenising the data 

In [240]:
len(y.unique())

15

In [266]:
from sklearn.preprocessing import OneHotEncoder

In [267]:
ohe=OneHotEncoder(Y)

In [287]:
Y=y.to_frame()

In [288]:
y_tf=pd.DataFrame(ohe.fit_transform(Y).toarray())

In [289]:
y_tf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [290]:
#map, cachec, shuffle,batch,prefect from tensorslice,
data=tf.data.Dataset.from_tensor_slices((vect_txt,y_tf))
data=data.cache()
data=data.shuffle(160000) #buffer size
data=data.batch(16) # batch of series of 16 sample
data=data.prefetch(8) #helps prevent bottle necks

In [ ]:
# text in vect format and y labels

In [291]:
batch_X,batch_y=data.as_numpy_iterator().next()

In [292]:
batch_X.shape,batch_y.shape

((16, 1800), (16, 15))

In [293]:
train_tf=data.take(int(len(data)*0.7))
val=data.skip(int(len(data)*0.7)).take(int(len(data)*0.2))
test=data.skip(int(len(data)*0.9)).take(int(len(data)*0.1))

In [294]:
len(train_tf),len(val),len(test),len(batch_X),len(data)

(3747, 1070, 535, 16, 5354)

In [225]:
train_gen=train_tf.as_numpy_iterator().next()

In [227]:
train_gen.next()

(array([[  12,  630, 1015, ...,    0,    0,    0],
        [   2,   17, 2699, ...,    0,    0,    0],
        [  12,  649,  145, ...,    0,    0,    0],
        ...,
        [   2, 4163,    7, ...,    0,    0,    0],
        [  12,   88,   10, ...,    0,    0,    0],
        [   2,  858,   27, ...,    0,    0,    0]], dtype=int64),
 array([b'Acne', b'Depression', b'Acne', b'Acne', b'Bipolar Disorde',
        b'Depression', b'Birth Control', b'Obesity',
        b'Vaginal Yeast Infection', b'Depression', b'Birth Control',
        b'Birth Control', b'Abnormal Uterine Bleeding', b'Birth Control',
        b'Birth Control', b'Diabetes, Type 2'], dtype=object))

# deep learning model

In [254]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dropout, Bidirectional,Embedding,Flatten

In [299]:
model=Sequential()
model.add(Embedding(20000+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Flatten())
model.add(Dense(15,activation='softmax')) #final layer

In [303]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [304]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 32)          640032    
                                                                 
 bidirectional_4 (Bidirectio  (None, 64)               16640     
 nal)                                                            
                                                                 
 dense_19 (Dense)            (None, 128)               8320      
                                                                 
 dense_20 (Dense)            (None, 256)               33024     
                                                                 
 dense_21 (Dense)            (None, 128)               32896     
                                                                 
 dense_22 (Dense)            (None, 128)               16512     
                                                      

In [305]:
model.fit(train_tf,epochs=1,validation_data=val)

3747/3747 [==============================] - 6781s 2s/step - loss: 0.9018 - accuracy: 0.6849 - val_loss: 0.5159 - val_accuracy: 0.8171


In [309]:
input_txt=vect(['i feel very cranky and sleepy all day'])

In [310]:
model.predict(np.array(input_txt))

1/1 [==============================] - 19s 19s/step


array([[0.07899219, 0.00483116, 0.00858389, 0.07704106, 0.19005755,
        0.01366973, 0.12712485, 0.01565679, 0.0068479 , 0.1376945 ,
        0.23116092, 0.01828908, 0.06252836, 0.00963321, 0.01788883]],
      dtype=float32)

In [314]:
#11 as 0.23 score
Y['condition'][10] # that is a some what a bad prediction

'Insomnia'

In [312]:
res=model.predict(np.expand_dims(input_txt,0))

In [ ]:
tbatch_X,tbatch_y=test.as_numpy_iterator().next()

In [ ]:
model.predict(tbatch_X[:10])

In [ ]:
(model.predict(tbatch_X)>0.5)astype(int)

In [306]:
ml_df['final_review'][0]

'My son halfway fourth week Intuniv We became concerned began last week started taking highest dose For two day could hardly get bed wa cranky slept nearly hour drive home school vacation unusual I called doctor Monday morning said stick day See school getting morning The last two day problem free He MUCH agreeable ever He le emotional good thing le cranky He remembering thing Overall behavior better We tried many different medication far effective'